<a href="https://colab.research.google.com/github/smartkhadka/Computer-Vision-/blob/main/Customdatasetipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset
from PIL import Image
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms

In [ ]:
class ClassificationDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Example transforms
img_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

# Usage
dataset_cls = ClassificationDataset(image_paths, labels, transform=img_transform)

In [ ]:
class SegmentationDataset(Dataset):
    def __init__(self, image_paths, mask_paths, transform_img=None, transform_mask=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform_img = transform_img
        self.transform_mask = transform_mask

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        mask = Image.open(self.mask_paths[idx]).convert("L")

        if self.transform_img:
            image = self.transform_img(image)
        if self.transform_mask:
            mask = self.transform_mask(mask)

        return image, mask

# Example transforms
img_transform = transforms.Compose([
    transforms.Resize((256,256), interpolation=Image.BILINEAR),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

mask_transform = transforms.Compose([
    transforms.Resize((256,256), interpolation=Image.NEAREST),
    transforms.ToTensor()
])

# Usage
dataset_seg = SegmentationDataset(image_paths, mask_paths,
                                  transform_img=img_transform,
                                  transform_mask=mask_transform)


In [ ]:



class DetectionDataset(Dataset):
    def __init__(self, image_paths, annotations, transform=None):
        self.image_paths = image_paths
        self.annotations = annotations  # list of dicts {"boxes":..., "labels":...}
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert("RGB")
        target = self.annotations[idx]  # dict with "boxes" and "labels"

        if self.transform:
            image, target = self.transform(image, target)

        return image, target

# Example transform (using torchvision detection API style)
import torchvision.transforms as T

def detection_transform(image, target):
    transform = T.Compose([
        T.Resize((300,300)),
        T.ToTensor()
    ])
    image = transform(image)
    # target stays as dict, but you may need to scale boxes if resizing
    return image, target

# Usage
dataset_det = DetectionDataset(image_paths, annotations, transform=detection_transform)

